In [ ]:
%load_ext autoreload
%autoreload 2

from exchange_data.data import OrderBookFrame
from exchange_data.data.backtest import Backtest
import pandas as pd
import numpy as np
import alog
import plotly.graph_objs as go
import plotly.express as px
import plotly as py
import plotly.offline as ofl
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)
params = {
 'cache': False,
 'database_name': 'binance_futures',
 'depth': 32,
 'futures': False,
 'group_by': '1m',
 'interval': '3d',
 'max_volume_quantile': 0.99,
 'offset_interval': '0h',
 'plot': False,
 'round_decimals': 2,
 'sequence_length': 48,
 'symbol': 'UNFIUSDT',
 'window_size': '1h'
}

df = Backtest(**params).frame

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2023-11-20 12:26:33 INFO  [data.backtest:21] 
{'cache': False,
 'database_name': 'binance_futures',
 'depth': 32,
 'futures': False,
 'group_by': '1m',
 'interval': '3d',
 'max_volume_quantile': 0.99,
 'offset_interval': '0h',
 'plot': False,
 'round_decimals': 2,
 'sequence_length': 48,
 'symbol': 'UNFIUSDT',
 'window_size': '1h'}
2023-11-20 12:26:33 DEBUG [exchange_data._database:64] SELECT last(*) AS data FROM UNFIUSDT_OrderBookFrame WHERE time >= '2023-11-17 17:38:33.231105' AND time <= '2023-11-17 18:38:33.231105' GROUP BY time(1m);
2023-11-20 12:26:33 DEBUG [exchange_data._database:64] SELECT last(*) AS data FROM UNFIUSDT_OrderBookFrame WHERE time >= '2023-11-17 18:38:33.231105' AND time <= '2023-11-17 19:38:33.231105' GROUP BY time(1m);
2023-11-20 12:26:33 DEBUG [exchange_data._database:64] SELECT last(*) AS data FROM UNFIUSDT_OrderBookFrame WHERE time >= '2023-11-17 19:38:33.231105' AND time <= '2023-11-17 20:38:33.231105' GROUP BY time(1m);
2023-11-20 12:26:33 DEBUG [exchange_

In [ ]:
df=df.loc[:, df.columns!='orderbook_img']
df

In [ ]:
capital = 1
df['capital'] = 0
df_len = df.shape[0]
capital_ls = []
price_in = None

for ix in range(0, df_len):
    if ix < df_len - 1:
        row = df.iloc[ix]
        next_row = df.iloc[ix + 1]
        best_ask = row['best_ask']
        
        if row['prediction'] == 1:
            if price_in == None:
                price_in = row['best_bid']
        else:
            if price_in:
                pnl = (best_ask - price_in) / price_in
                capital = capital + (capital * pnl)
                
            price_in = None
            
    #alog.info(capital)
    capital_ls.append(capital)

df['capital'] = np.asarray(capital_ls)

df

In [ ]:
px.line(df, y='capital')